In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [2]:
footballer_df = pd.read_csv('footballer_stats_from_R.csv', low_memory = False)

In [3]:
footballer_fuzzy = pd.read_csv('footballer_stats_from_fuzzy.csv', low_memory = False)

In [4]:
keeper_df = pd.read_csv('keeper_stats_from_R.csv')

In [5]:
price_value = pd.read_csv("transfermarkt_price.csv")

In [6]:
footballer_df

,Season_End_Year_x,Squad_x,Comp_x,Player_x,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,PrgC_Carries,Final_Third_Carries,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk
0,2018,Alavés,La Liga,Martin Agirregabiria,ESP,DF,21.0,1996.0,21,20,...,24.0,13.0,6.0,20.0,10.0,483.0,54.0,https://fbref.com/en/players/355c883a/Martin-A...,2018Alavés,14
1,2018,Alavés,La Liga,Alexis,ESP,DF,31.0,1985.0,28,25,...,5.0,3.0,2.0,8.0,3.0,389.0,11.0,https://fbref.com/en/players/db0cd1ae/Alexis,2018Alavés,14
2,2018,Alavés,La Liga,Burgui,ESP,"MF,FW",23.0,1993.0,23,12,...,35.0,17.0,14.0,23.0,17.0,348.0,75.0,https://fbref.com/en/players/45e304c8/Burgui,2018Alavés,14
3,2018,Alavés,La Liga,Ermedin Demirović,BIH,FW,19.0,1998.0,3,2,...,2.0,2.0,2.0,4.0,4.0,42.0,11.0,https://fbref.com/en/players/ed79b7d3/Ermedin-...,2018Alavés,14
4,2018,Alavés,La Liga,Adrián Diéguez,ESP,DF,21.0,1996.0,6,6,...,10.0,4.0,1.0,3.0,3.0,103.0,3.0,https://fbref.com/en/players/70438d35/Adrian-D...,2018Alavés,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17535,2024,Wolves,Premier League,José Sá,POR,GK,31-056,1993.0,26,26,...,0.0,1.0,1.0,0.0,1.0,328.0,0.0,https://fbref.com/en/players/903b6e8b/Jose-Sa,2024Wolves,9
17536,2024,Wolves,Premier League,Pablo Sarabia,ESP,"MF,FW",31-307,1992.0,21,15,...,33.0,40.0,8.0,28.0,16.0,605.0,105.0,https://fbref.com/en/players/9744ff80/Pablo-Sa...,2024Wolves,9
17537,2024,Wolves,Premier League,Nélson Semedo,POR,"DF,MF",30-118,1993.0,27,27,...,79.0,48.0,10.0,42.0,25.0,978.0,113.0,https://fbref.com/en/players/d04b94db/Nelson-S...,2024Wolves,9
17538,2024,Wolves,Premier League,Fábio Silva,POR,FW,21-238,2002.0,8,3,...,5.0,5.0,1.0,12.0,4.0,69.0,13.0,https://fbref.com/en/players/e7aa9d7c/Fabio-Silva,2024Wolves,9


In [7]:
footballer_df.rename(columns={'Season_End_Year_x': 'Season'}, inplace=True)
footballer_df.rename(columns={'Player_x': 'Name'}, inplace=True)

In [8]:
keeper_df.rename(columns={'Season_End_Year': 'Season'}, inplace=True)
keeper_df.rename(columns={'Player': 'Name'}, inplace=True)

### We need to reduce the season by 1, because in the dataset it was recorded as end season not the start season

In [9]:
footballer_df['Season'] = footballer_df['Season']-1
keeper_df['Season'] = keeper_df['Season']-1

In [10]:
price_value.rename(columns={'Year': 'Season'}, inplace=True)

In [11]:
price_value

,Name,Value,Season,Position
0,Ederson,€40.00m,2023,Goalkeeper
1,Stefan Ortega,€9.00m,2023,Goalkeeper
2,Scott Carson,€200k,2023,Goalkeeper
3,Rúben Dias,€80.00m,2023,Centre-Back
4,Josko Gvardiol,€75.00m,2023,Centre-Back
...,...,...,...,...
28682,Roy Tol,€50k,2017,Right Winger
28683,Enzo Stroo,€250k,2017,Centre-Forward
28684,Teije ten Den,€150k,2017,Centre-Forward
28685,Jordy Hilterman,€50k,2017,Centre-Forward


### Combine normal footballer dataframe and dataframe from Fuzzy algorithm, then replace the first value

In [12]:
footballer_df = pd.concat([footballer_df, footballer_fuzzy], ignore_index=True)

In [13]:
footballer_df.drop_duplicates(subset='ID',keep= 'last', inplace=True)

In [14]:
footballer_df[footballer_df['Name']== "Heung-min Son"]

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,PrgC_Carries,Final_Third_Carries,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk
17918,2017,Tottenham,Premier League,Heung-min Son,KOR,FW,25.0,1992.0,37.0,27.0,...,111.0,46.0,68.0,65.0,57.0,1067.0,339.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2018Tottenham,3.0
18324,2018,Tottenham,Premier League,Heung-min Son,KOR,"FW,MF",26.0,1992.0,31.0,23.0,...,105.0,64.0,59.0,60.0,43.0,841.0,201.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2019Tottenham,4.0
18729,2019,Tottenham,Premier League,Heung-min Son,KOR,FW,27.0,1992.0,30.0,28.0,...,90.0,49.0,56.0,66.0,57.0,853.0,215.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2020Tottenham,6.0
19153,2020,Tottenham,Premier League,Heung-min Son,KOR,FW,28.0,1992.0,37.0,36.0,...,88.0,46.0,37.0,59.0,50.0,1091.0,266.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2021Tottenham,7.0
19545,2021,Tottenham,Premier League,Heung-min Son,KOR,"MF,FW",29.0,1992.0,35.0,35.0,...,116.0,68.0,50.0,75.0,46.0,1078.0,242.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2022Tottenham,4.0
19927,2022,Tottenham,Premier League,Heung-min Son,KOR,"MF,FW",30.0,1992.0,36.0,33.0,...,82.0,60.0,31.0,64.0,55.0,860.0,196.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2023Tottenham,8.0
20268,2023,Tottenham,Premier League,Heung-min Son,KOR,FW,31-249,1992.0,24.0,23.0,...,70.0,27.0,42.0,36.0,25.0,717.0,228.0,https://fbref.com/en/players/92e7e919/Son-Heun...,2024Tottenham,5.0


# Merge Footballer

In [15]:
price_value

,Name,Value,Season,Position
0,Ederson,€40.00m,2023,Goalkeeper
1,Stefan Ortega,€9.00m,2023,Goalkeeper
2,Scott Carson,€200k,2023,Goalkeeper
3,Rúben Dias,€80.00m,2023,Centre-Back
4,Josko Gvardiol,€75.00m,2023,Centre-Back
...,...,...,...,...
28682,Roy Tol,€50k,2017,Right Winger
28683,Enzo Stroo,€250k,2017,Centre-Forward
28684,Teije ten Den,€150k,2017,Centre-Forward
28685,Jordy Hilterman,€50k,2017,Centre-Forward


In [16]:
conditions = ['Season','Name']

In [17]:
footballer_with_price_df= pd.merge(footballer_df, price_value , on=conditions)

In [18]:
# footballer_with_price_df.drop_duplicates(inplace = True)

In [19]:
len(footballer_with_price_df)

13400

In [20]:
footballer_with_price_df

,Season,Squad_x,Comp_x,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
0,2017,Alavés,La Liga,Burgui,ESP,"MF,FW",23.0,1993.0,23.0,12.0,...,14.0,23.0,17.0,348.0,75.0,https://fbref.com/en/players/45e304c8/Burgui,2018Alavés,14.0,€1.80m,Left Winger
1,2017,Alavés,La Liga,Adrián Diéguez,ESP,DF,21.0,1996.0,6.0,6.0,...,1.0,3.0,3.0,103.0,3.0,https://fbref.com/en/players/70438d35/Adrian-D...,2018Alavés,14.0,€400k,Centre-Back
2,2017,Alavés,La Liga,Rubén Duarte,ESP,DF,21.0,1995.0,24.0,24.0,...,2.0,12.0,13.0,466.0,48.0,https://fbref.com/en/players/1fe0492d/Ruben-Du...,2018Alavés,14.0,€2.00m,Left-Back
3,2017,Alavés,La Liga,Rodrigo Ely,BRA,DF,23.0,1993.0,31.0,30.0,...,0.0,9.0,3.0,635.0,7.0,https://fbref.com/en/players/fd47b4f2/Rodrigo-Ely,2018Alavés,14.0,€6.00m,Centre-Back
4,2017,Alavés,La Liga,Manu García,ESP,MF,31.0,1986.0,30.0,27.0,...,1.0,28.0,28.0,761.0,48.0,https://fbref.com/en/players/05a4465d/Manu-Garcia,2018Alavés,14.0,€1.80m,Central Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13395,2023,Wolfsburg,Bundesliga,Jakub Kaminski,POL,"FW,MF",21-282,2002.0,13.0,3.0,...,5.0,3.0,9.0,99.0,31.0,https://fbref.com/en/players/202b3b68/Jakub-Ka...,2024Wolfsburg,13.0,€7.00m,Left Winger
13396,2023,Wolfsburg,Bundesliga,Joakim Maehle,DEN,DF,26-298,1997.0,23.0,22.0,...,7.0,25.0,12.0,811.0,105.0,https://fbref.com/en/players/c459a650/Joakim-M...,2024Wolfsburg,13.0,€14.00m,Right-Back
13397,2023,Wolfsburg,Bundesliga,Cédric Zesiger,SUI,DF,25-263,1998.0,15.0,12.0,...,0.0,1.0,2.0,597.0,1.0,https://fbref.com/en/players/2bab583c/Cedric-Z...,2024Wolfsburg,13.0,€6.00m,Centre-Back
13398,2023,Wolves,Premier League,Rayan Aït-Nouri,ALG,"DF,MF",22-281,2001.0,24.0,21.0,...,7.0,34.0,35.0,736.0,78.0,https://fbref.com/en/players/9b398aea/Rayan-Ai...,2024Wolves,9.0,€32.00m,Left-Back


In [21]:
footballer_with_price_df.rename(columns={'Comp_x': 'Compe'}, inplace=True)

In [22]:
footballer_with_price_df['Position'].unique()

array(['Left Winger', 'Centre-Back', 'Left-Back', 'Central Midfield',
       'Right Winger', 'Right-Back', 'Goalkeeper', 'Attacking Midfield',
       'Centre-Forward', 'Defensive Midfield', 'Right Midfield',
       'Left Midfield', 'Second Striker'], dtype=object)

In [23]:
footballer_with_price_df[footballer_with_price_df['Position']=='Goalkeeper']

,Season,Squad_x,Compe,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
12,2017,Alavés,La Liga,Fernando Pacheco,ESP,GK,25.0,1992.0,38.0,37.0,...,0.0,3.0,0.0,298.0,0.0,https://fbref.com/en/players/97852360/Fernando...,2018Alavés,14.0,€9.00m,Goalkeeper
16,2017,Alavés,La Liga,Antonio Sivera,ESP,GK,20.0,1996.0,1.0,1.0,...,0.0,0.0,0.0,4.0,0.0,https://fbref.com/en/players/9970da54/Antonio-...,2018Alavés,14.0,€1.00m,Goalkeeper
36,2017,Arsenal,Premier League,David Ospina,COL,GK,28.0,1988.0,5.0,4.0,...,1.0,0.0,0.0,50.0,0.0,https://fbref.com/en/players/82b1198a/David-Os...,2018Arsenal,6.0,€5.00m,Goalkeeper
45,2017,Atalanta,Serie A,Etrit Berisha,ALB,GK,28.0,1989.0,31.0,31.0,...,0.0,1.0,0.0,286.0,0.0,https://fbref.com/en/players/ee833569/Etrit-Be...,2018Atalanta,7.0,€7.00m,Goalkeeper
51,2017,Atalanta,Serie A,Pierluigi Gollini,ITA,GK,22.0,1995.0,7.0,7.0,...,0.0,0.0,0.0,84.0,0.0,https://fbref.com/en/players/f0f7f62f/Pierluig...,2018Atalanta,7.0,€3.50m,Goalkeeper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13370,2023,Sevilla,La Liga,Marko Dmitrovic,SRB,GK,32-049,1992.0,13.0,13.0,...,0.0,0.0,0.0,190.0,0.0,https://fbref.com/en/players/ac295d9e/Marko-Dm...,2024Sevilla,14.0,€1.00m,Goalkeeper
13373,2023,Sheffield Utd,Premier League,Ivo Grbic,CRO,GK,28-055,1996.0,4.0,4.0,...,0.0,0.0,0.0,35.0,0.0,https://fbref.com/en/players/3f034750/Ivo-Grbic,2024Sheffield Utd,20.0,€3.50m,Goalkeeper
13386,2023,Union Berlin,Bundesliga,Frederik Rönnow,DEN,GK,31-222,1992.0,24.0,24.0,...,1.0,1.0,0.0,517.0,0.0,https://fbref.com/en/players/a70b8345/Frederik...,2024Union Berlin,14.0,€5.50m,Goalkeeper
13391,2023,Werder Bremen,Bundesliga,Jiri Pavlenka,CZE,GK,31-334,1992.0,7.0,7.0,...,0.0,1.0,0.0,94.0,0.0,https://fbref.com/en/players/0a1cb833/Jiri-Pav...,2024Werder Bremen,10.0,€1.50m,Goalkeeper


In [24]:
footballer_with_price_df.isnull().sum()

Season        0
Squad_x       0
Compe         0
Name          0
Nation_x      7
             ..
Url           0
Squad_Year    0
Rk            0
Value         0
Position      0
Length: 172, dtype: int64

# Merge Keeper

In [25]:
keeper_with_price_df= pd.merge(keeper_df, price_value , on=conditions)

In [26]:
keeper_with_price_df.drop_duplicates(inplace = True)

In [27]:
keeper_with_price_df.dropna(inplace= True)

In [28]:
keeper_with_price_df

,Season,Squad,Comp,Name,Nation,Pos,Age,Born,Mins_Per_90,GA_Goals,...,Stp_Crosses,Stp_percent_Crosses,#OPA_Sweeper,#OPA_per_90_Sweeper,AvgDist_Sweeper,Url,Squad_Year,Rk,Value,Position
0,2017,Alavés,La Liga,Fernando Pacheco,ESP,GK,25,1992,37.3,50.0,...,17.0,2.6,19.0,0.51,12.5,https://fbref.com/en/players/97852360/Fernando...,2018Alavés,14,€9.00m,Goalkeeper
2,2017,Arsenal,Premier League,David Ospina,COL,GK,28,1988,4.2,3.0,...,6.0,7.3,7.0,1.65,13.1,https://fbref.com/en/players/82b1198a/David-Os...,2018Arsenal,6,€5.00m,Goalkeeper
3,2017,Atalanta,Serie A,Etrit Berisha,ALB,GK,28,1989,31.0,34.0,...,18.0,4.5,25.0,0.81,14.1,https://fbref.com/en/players/ee833569/Etrit-Be...,2018Atalanta,7,€7.00m,Goalkeeper
4,2017,Atalanta,Serie A,Pierluigi Gollini,ITA,GK,22,1995,6.9,4.0,...,2.0,2.2,5.0,0.72,15.7,https://fbref.com/en/players/f0f7f62f/Pierluig...,2018Atalanta,7,€3.50m,Goalkeeper
6,2017,Athletic Club,La Liga,Kepa Arrizabalaga,ESP,GK,22,1994,30.0,43.0,...,23.0,5.8,22.0,0.73,13.4,https://fbref.com/en/players/28d596a0/Kepa-Arr...,2018Athletic Club,16,€20.00m,Goalkeeper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,2023,West Ham,Premier League,Alphonse Areola,FRA,GK,31-015,1993,24.5,40.0,...,22.0,5.7,7.0,0.29,9.9,https://fbref.com/en/players/2f965a72/Alphonse...,2024West Ham,7,€10.00m,Goalkeeper
1052,2023,Wolfsburg,Bundesliga,Koen Casteels,BEL,GK,31-262,1992,21.0,33.0,...,21.0,7.0,37.0,1.76,16.4,https://fbref.com/en/players/db401046/Koen-Cas...,2024Wolfsburg,13,€8.00m,Goalkeeper
1053,2023,Wolfsburg,Bundesliga,Pavao Pervan,AUT,GK,36-121,1987,4.0,8.0,...,0.0,0.0,3.0,0.75,15.7,https://fbref.com/en/players/c89e8aae/Pavao-Pe...,2024Wolfsburg,13,€400k,Goalkeeper
1054,2023,Wolves,Premier League,Daniel Bentley,ENG,GK,30-244,1993,3.3,4.0,...,0.0,0.0,2.0,0.61,12.7,https://fbref.com/en/players/68db6358/Daniel-B...,2024Wolves,9,€2.00m,Goalkeeper


In [29]:
keeper_with_price_df[keeper_with_price_df['Position']!='Goalkeeper']

,Season,Squad,Comp,Name,Nation,Pos,Age,Born,Mins_Per_90,GA_Goals,...,Stp_Crosses,Stp_percent_Crosses,#OPA_Sweeper,#OPA_per_90_Sweeper,AvgDist_Sweeper,Url,Squad_Year,Rk,Value,Position
28,2017,Cagliari,Serie A,Rafael,BRA,GK,35,1982,8.8,11.0,...,4.0,2.6,3.0,0.34,9.5,https://fbref.com/en/players/dd06d56c/Rafael,2018Cagliari,16,€9.00m,Right-Back
61,2017,Lille,Ligue 1,Nicolas de Préville,FRA,"FW,MF",26,1991,3.2,1.0,...,0.0,0.0,1.0,3.33,37.0,https://fbref.com/en/players/b0547a87/Nicolas-...,2018Lille,17,€7.00m,Centre-Forward
84,2017,Málaga,La Liga,Roberto,ESP,GK,31,1986,34.0,51.0,...,28.0,5.8,20.0,0.59,11.8,https://fbref.com/en/players/ba3ca78f/Roberto,2018Málaga,20,€250k,Centre-Forward
313,2019,Cagliari,Serie A,Rafael,BRA,GK,37,1982,5.0,11.0,...,3.0,3.1,3.0,0.59,11.1,https://fbref.com/en/players/dd06d56c/Rafael,2020Cagliari,14,€5.50m,Right-Back
631,2021,Espanyol,La Liga,Diego López,ESP,GK,39,1981,35.3,49.0,...,35.0,5.7,43.0,1.22,13.7,https://fbref.com/en/players/73683fa9/Diego-Lopez,2022Espanyol,14,-,Left Winger
632,2021,Espanyol,La Liga,Diego López,ESP,GK,39,1981,35.3,49.0,...,35.0,5.7,43.0,1.22,13.7,https://fbref.com/en/players/73683fa9/Diego-Lopez,2022Espanyol,14,-,Attacking Midfield
738,2022,Almería,La Liga,Fernando,ESP,GK,32,1990,36.5,61.0,...,39.0,6.4,26.0,0.71,12.4,https://fbref.com/en/players/dabb2827/Fernando,2023Almería,17,€3.00m,Defensive Midfield
869,2022,Rayo Vallecano,La Liga,Diego López,ESP,GK,40,1981,1.1,4.0,...,0.0,0.0,2.0,1.88,17.0,https://fbref.com/en/players/73683fa9/Diego-Lopez,2023Rayo Vallecano,11,€1.50m,Left Winger


# Dapetin Kolom Tertentu

# Coba di clean lagi footballer_df_ready nya termasuk formatting

In [30]:
footballer_with_price_df.dropna(inplace= True)

In [31]:
# coba di benerin rank nya

In [32]:
# footballer_df_ready['Rank'] = footballer_df_ready['Rank'].str.extract('(\d+)').astype(int)

In [33]:
footballer_with_price_df

,Season,Squad_x,Compe,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
0,2017,Alavés,La Liga,Burgui,ESP,"MF,FW",23.0,1993.0,23.0,12.0,...,14.0,23.0,17.0,348.0,75.0,https://fbref.com/en/players/45e304c8/Burgui,2018Alavés,14.0,€1.80m,Left Winger
3,2017,Alavés,La Liga,Rodrigo Ely,BRA,DF,23.0,1993.0,31.0,30.0,...,0.0,9.0,3.0,635.0,7.0,https://fbref.com/en/players/fd47b4f2/Rodrigo-Ely,2018Alavés,14.0,€6.00m,Centre-Back
4,2017,Alavés,La Liga,Manu García,ESP,MF,31.0,1986.0,30.0,27.0,...,1.0,28.0,28.0,761.0,48.0,https://fbref.com/en/players/05a4465d/Manu-Garcia,2018Alavés,14.0,€1.80m,Central Midfield
5,2017,Alavés,La Liga,Ibai Gómez,ESP,"MF,FW",27.0,1989.0,34.0,26.0,...,9.0,49.0,52.0,788.0,117.0,https://fbref.com/en/players/9d2a3d5f/Ibai-Gomez,2018Alavés,14.0,€3.50m,Left Winger
6,2017,Alavés,La Liga,Munir El Haddadi,MAR,FW,21.0,1995.0,33.0,28.0,...,14.0,90.0,87.0,779.0,173.0,https://fbref.com/en/players/8696bc90/Munir-El...,2018Alavés,14.0,€12.00m,Right Winger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13395,2023,Wolfsburg,Bundesliga,Jakub Kaminski,POL,"FW,MF",21-282,2002.0,13.0,3.0,...,5.0,3.0,9.0,99.0,31.0,https://fbref.com/en/players/202b3b68/Jakub-Ka...,2024Wolfsburg,13.0,€7.00m,Left Winger
13396,2023,Wolfsburg,Bundesliga,Joakim Maehle,DEN,DF,26-298,1997.0,23.0,22.0,...,7.0,25.0,12.0,811.0,105.0,https://fbref.com/en/players/c459a650/Joakim-M...,2024Wolfsburg,13.0,€14.00m,Right-Back
13397,2023,Wolfsburg,Bundesliga,Cédric Zesiger,SUI,DF,25-263,1998.0,15.0,12.0,...,0.0,1.0,2.0,597.0,1.0,https://fbref.com/en/players/2bab583c/Cedric-Z...,2024Wolfsburg,13.0,€6.00m,Centre-Back
13398,2023,Wolves,Premier League,Rayan Aït-Nouri,ALG,"DF,MF",22-281,2001.0,24.0,21.0,...,7.0,34.0,35.0,736.0,78.0,https://fbref.com/en/players/9b398aea/Rayan-Ai...,2024Wolves,9.0,€32.00m,Left-Back


In [34]:
footballer_with_price_df.dtypes

Season          int64
Squad_x        object
Compe          object
Name           object
Nation_x       object
               ...   
Url            object
Squad_Year     object
Rk            float64
Value          object
Position       object
Length: 172, dtype: object

In [35]:
footballer_with_price_df['Compe'].unique()

array(['La Liga', 'Ligue 1', 'Premier League', 'Serie A', 'Bundesliga'],
      dtype=object)

### Hilangin compe di luar big 5

In [36]:
allowed_compe = ['Premier League', 'La Liga', 'Serie A', 'Bundesliga', 'Ligue 1']

In [37]:
top_league_footballer_df = footballer_with_price_df[footballer_with_price_df['Compe'].isin(allowed_compe)]

In [38]:
top_league_footballer_df

,Season,Squad_x,Compe,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
0,2017,Alavés,La Liga,Burgui,ESP,"MF,FW",23.0,1993.0,23.0,12.0,...,14.0,23.0,17.0,348.0,75.0,https://fbref.com/en/players/45e304c8/Burgui,2018Alavés,14.0,€1.80m,Left Winger
3,2017,Alavés,La Liga,Rodrigo Ely,BRA,DF,23.0,1993.0,31.0,30.0,...,0.0,9.0,3.0,635.0,7.0,https://fbref.com/en/players/fd47b4f2/Rodrigo-Ely,2018Alavés,14.0,€6.00m,Centre-Back
4,2017,Alavés,La Liga,Manu García,ESP,MF,31.0,1986.0,30.0,27.0,...,1.0,28.0,28.0,761.0,48.0,https://fbref.com/en/players/05a4465d/Manu-Garcia,2018Alavés,14.0,€1.80m,Central Midfield
5,2017,Alavés,La Liga,Ibai Gómez,ESP,"MF,FW",27.0,1989.0,34.0,26.0,...,9.0,49.0,52.0,788.0,117.0,https://fbref.com/en/players/9d2a3d5f/Ibai-Gomez,2018Alavés,14.0,€3.50m,Left Winger
6,2017,Alavés,La Liga,Munir El Haddadi,MAR,FW,21.0,1995.0,33.0,28.0,...,14.0,90.0,87.0,779.0,173.0,https://fbref.com/en/players/8696bc90/Munir-El...,2018Alavés,14.0,€12.00m,Right Winger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13395,2023,Wolfsburg,Bundesliga,Jakub Kaminski,POL,"FW,MF",21-282,2002.0,13.0,3.0,...,5.0,3.0,9.0,99.0,31.0,https://fbref.com/en/players/202b3b68/Jakub-Ka...,2024Wolfsburg,13.0,€7.00m,Left Winger
13396,2023,Wolfsburg,Bundesliga,Joakim Maehle,DEN,DF,26-298,1997.0,23.0,22.0,...,7.0,25.0,12.0,811.0,105.0,https://fbref.com/en/players/c459a650/Joakim-M...,2024Wolfsburg,13.0,€14.00m,Right-Back
13397,2023,Wolfsburg,Bundesliga,Cédric Zesiger,SUI,DF,25-263,1998.0,15.0,12.0,...,0.0,1.0,2.0,597.0,1.0,https://fbref.com/en/players/2bab583c/Cedric-Z...,2024Wolfsburg,13.0,€6.00m,Centre-Back
13398,2023,Wolves,Premier League,Rayan Aït-Nouri,ALG,"DF,MF",22-281,2001.0,24.0,21.0,...,7.0,34.0,35.0,736.0,78.0,https://fbref.com/en/players/9b398aea/Rayan-Ai...,2024Wolves,9.0,€32.00m,Left-Back


In [39]:
top_league_footballer_df[top_league_footballer_df['Name']=='Lionel Messi']

,Season,Squad_x,Compe,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,CPA_Carries,Mis_Carries,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position
136,2017,Barcelona,La Liga,Lionel Messi,ARG,FW,30.0,1987.0,36.0,32.0,...,95.0,52.0,107.0,2076.0,263.0,https://fbref.com/en/players/d70ce98e/Lionel-M...,2018Barcelona,1.0,€180.00m,Right Winger
1707,2018,Barcelona,La Liga,Lionel Messi,ARG,FW,31.0,1987.0,34.0,29.0,...,52.0,63.0,78.0,2017.0,206.0,https://fbref.com/en/players/d70ce98e/Lionel-M...,2019Barcelona,1.0,€150.00m,Right Winger
3394,2019,Barcelona,La Liga,Lionel Messi,ARG,FW,32.0,1987.0,33.0,32.0,...,63.0,69.0,78.0,2175.0,252.0,https://fbref.com/en/players/d70ce98e/Lionel-M...,2020Barcelona,2.0,€112.00m,Right Winger
5102,2020,Barcelona,La Liga,Lionel Messi,ARG,"FW,MF",33.0,1987.0,35.0,33.0,...,65.0,57.0,87.0,2481.0,263.0,https://fbref.com/en/players/d70ce98e/Lionel-M...,2021Barcelona,3.0,€80.00m,Right Winger
8100,2021,Paris S-G,Ligue 1,Lionel Messi,ARG,"FW,MF",34.0,1987.0,26.0,24.0,...,25.0,47.0,41.0,1667.0,125.0,https://fbref.com/en/players/d70ce98e/Lionel-M...,2022Paris S-G,1.0,€50.00m,Right Winger
9997,2022,Paris S-G,Ligue 1,Lionel Messi,ARG,"FW,MF",35.0,1987.0,32.0,32.0,...,42.0,52.0,77.0,2042.0,220.0,https://fbref.com/en/players/d70ce98e/Lionel-M...,2023Paris S-G,1.0,€35.00m,Right Winger


In [40]:
def total_teams_in_league(value):
    if value == 'Premier League' or value == "La Liga" or value =="Serie A":
        return 20
    elif value == "Bundesliga" or value == "Ligue 1" :
        return 18

In [41]:
top_league_footballer_df['total_teams_in_league'] =top_league_footballer_df['Compe'].apply(total_teams_in_league)

# Change Value or Price into a real number value

In [42]:
top_league_footballer_df = top_league_footballer_df[top_league_footballer_df['Value']!='-']

In [43]:
def convert_to_numeric(value):
    if 'm' in value:
        return float(value.replace('€', '').replace('m', '')) * 1e6
    elif 'k' in value:
        return float(value.replace('€', '').replace('k', '')) * 1e3
    else:
        return float(value.replace('€', ''))

In [44]:
top_league_footballer_df['Price'] = top_league_footballer_df['Value'].apply(convert_to_numeric)

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/369626883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_league_footballer_df['Price'] = top_league_footballer_df['Value'].apply(convert_to_numeric)


In [45]:
keeper_with_price_df = keeper_with_price_df[keeper_with_price_df['Value']!='-']

In [46]:
keeper_with_price_df['Price'] = keeper_with_price_df['Value'].apply(convert_to_numeric)

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/3571880830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_with_price_df['Price'] = keeper_with_price_df['Value'].apply(convert_to_numeric)


In [47]:
keeper_with_price_df['Age'] = keeper_with_price_df['Season']-keeper_with_price_df['Born']

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/642761138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_with_price_df['Age'] = keeper_with_price_df['Season']-keeper_with_price_df['Born']


In [48]:
top_league_footballer_df[top_league_footballer_df['Price']>=190000000]

,Season,Squad_x,Compe,Name,Nation_x,Pos_x,Age_x,Born_x,MP_Playing,Starts_Playing,...,Dis_Carries,Rec_Receiving,PrgR_Receiving,Url,Squad_Year,Rk,Value,Position,total_teams_in_league,Price
2709,2018,Paris S-G,Ligue 1,Kylian Mbappé,FRA,FW,19.0,1998.0,29.0,24.0,...,44.0,995.0,311.0,https://fbref.com/en/players/42fd9c7f/Kylian-M...,2019Paris S-G,1.0,€200.00m,Centre-Forward,18,200000000.0


In [49]:
top_league_footballer_df['Price'].describe()

count    9.166000e+03
mean     1.488260e+07
std      1.932443e+07
min      5.000000e+04
25%      3.000000e+06
50%      8.000000e+06
75%      2.000000e+07
max      2.000000e+08
Name: Price, dtype: float64

In [50]:
top_league_footballer_df.to_csv("footballer_visualization.csv", index = False)

In [51]:
keeper_with_price_df.to_csv("keeper_visualization.csv", index = False)

### Data used for neural network

In [52]:
top_league_footballer_df['Age'] = top_league_footballer_df['Season']-top_league_footballer_df['Born_x']

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/4276332468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_league_footballer_df['Age'] = top_league_footballer_df['Season']-top_league_footballer_df['Born_x']


In [53]:
name = top_league_footballer_df['Name']
club = top_league_footballer_df['Squad_x']

In [54]:
columns_to_drop = ['Squad_x', 'Name','Nation_x','Pos_x','Age_x','Born_x','Url_x','ID',"Season_End_Year_y",'Squad_y',
                   'Comp_y', 'Player_y', 'Nation_y', 'Pos_y','Age_y','Born_y','Mins_Per_90_x','Url_y','Mins_Per_90_y','Ast_y',
                  'xG_Expected_y','npxG_Expected_y','Season_End_Year','Squad','Comp', 'Player','Nation','Pos','Born',
                  'Url','Squad_Year','Value','total_teams_in_league']
top_league_footballer_df.drop(columns=columns_to_drop, inplace=True)

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/3144312870.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_league_footballer_df.drop(columns=columns_to_drop, inplace=True)


In [55]:
top_league_footballer_df.to_csv("footballer_neural_network.csv", index = False)

In [56]:
keeper_with_price_df['Age'] = keeper_with_price_df['Season']-keeper_with_price_df['Born']

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/642761138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_with_price_df['Age'] = keeper_with_price_df['Season']-keeper_with_price_df['Born']


In [57]:
columns_to_drop_gk = ['Squad', 'Name','Nation','Pos', 'Born','Url', 'Value','Position']
keeper_with_price_df.drop(columns=columns_to_drop_gk, inplace=True)

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/2863278313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_with_price_df.drop(columns=columns_to_drop_gk, inplace=True)


In [58]:
keeper_with_price_df.to_csv("keeper_neural_network.csv", index = False)

### For SAP needs

In [59]:
def classify_positions(position):
    defender_positions = ['Centre-Back', 'Right-Back', 'Left-Back']
    midfielder_positions = ['Defensive Midfield', 'Central Midfield', 
                             'Attacking Midfield', 'Left Midfield', 'Right Midfield']
    attacker_positions = ['Centre-Forward', 'Second Striker', "Right Winger", 'Left Winger' ]

    if position in defender_positions:
        return 'Defender'
    elif position in midfielder_positions:
        return 'Midfielder'
    elif position in attacker_positions:
        return 'Attacker'
    else:
        return 'Goalkeeper'

In [60]:
top_league_footballer_df['Position_Category'] = top_league_footballer_df['Position'].apply(classify_positions)

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/1449609358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_league_footballer_df['Position_Category'] = top_league_footballer_df['Position'].apply(classify_positions)


In [61]:
top_league_footballer_df['Name']= name
top_league_footballer_df['Club']= club

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/617438440.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_league_footballer_df['Name']= name
C:\Users\russe\AppData\Local\Temp/ipykernel_2420/617438440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_league_footballer_df['Club']= club


In [62]:
top_league_footballer_df.to_csv("footballer_SAP.csv", index = False)

In [66]:
top_league_footballer_df[(top_league_footballer_df['Season']==2017) &(top_league_footballer_df['Compe']=='Premier League')]['Price'].describe()

count    2.210000e+02
mean     2.189932e+07
std      2.561965e+07
min      2.500000e+05
25%      7.000000e+06
50%      1.500000e+07
75%      2.500000e+07
max      1.500000e+08
Name: Price, dtype: float64

In [65]:
top_league_footballer_df.dropna(inplace=True)

C:\Users\russe\AppData\Local\Temp/ipykernel_2420/357417032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_league_footballer_df.dropna(inplace=True)
